<a href="https://colab.research.google.com/github/mablus/EFRAIM/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded_file = files.upload()
pdf_filename = list(uploaded_file.keys())[0]

Saving FP_PRIOC_SJKBSB.pdf to FP_PRIOC_SJKBSB.pdf


In [5]:
pdf_path = pdf_filename

In [3]:
!pip install PyMuPDF
!sudo apt-get install -y libcrypt-dev
!pip install PyPDF2

  Using cached PyMuPDF-1.23.3-cp310-none-manylinux2014_x86_64.whl (4.3 MB)
  Using cached PyMuPDFb-1.23.3-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (30.6 MB)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcrypt-dev is already the newest version (1:4.4.27-1).
libcrypt-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [10]:
import re
from google.colab import files
import PyPDF2

def recognize_flight_plan(text):
    keywords = ['flight number', 'origin', 'destination', 'alternate', 'orgn', 'org', 'dst' ,'dest', 'altrn', 'altn', 'mzfw', 'tow','ETOW','ELDW','EZFW','EPLD','MLDW','MZFW']

    for keyword in keywords:
        if re.search(keyword, text, re.IGNORECASE):
            return True
    return False

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

# Specify the path to your PDF file
pdf_path = '/content/FP_PRIOC_SJKBSB.pdf'
#pdf_path = input("Enter the path to the PDF file: ") caminho do site
pdf_text = extract_text_from_pdf(pdf_path)

if recognize_flight_plan(pdf_text):
    print("Recognized as a flight plan.")
else:
    print("Not recognized as a flight plan.")

def recognize_flight_plan(text):
    keywords = ['flight number', 'origin', 'destination', 'alternate', 'orgn', 'dest', 'altrn', 'altn', 'mzfw', 'tow']

    for keyword in keywords:
        if re.search(keyword, text, re.IGNORECASE):
            return True
    return False

def extract_fuel_data(text):
    fuel_data = {}

    fuel_pattern = r"TRIP (\d+) (\d+\.\d+) (\d+) (\d+) ETOW (\d+) MTOW (\d+)\s+ALTN/(\w{4}) (\d+) (\d+\.\d+) (\d+) (\d+) ELDW (\d+) MLDW (\d+)\s+RESERVE \(10%\) (\d+) (\d+\.\d+) EZFW (\d+) MZFW (\d+)\s+HOLDING (\d+) (\d+\.\d+) EPLD (\d+)\s+MIN FUEL REQ (\d+) (\d+\.\d+) OPNLWT (\d+)\s+CONTINGENCY"

    fuel_match = re.search(fuel_pattern, text)
    if fuel_match:
        fuel_data['trip_dist'] = int(fuel_match.group(1))
        fuel_data['trip_time'] = float(fuel_match.group(2))
        fuel_data['trip_fuel'] = int(fuel_match.group(3))
        fuel_data['planned_fuel'] = int(fuel_match.group(4))
        fuel_data['etow'] = int(fuel_match.group(5))
        fuel_data['mtow'] = int(fuel_match.group(6))
        fuel_data['altn'] = fuel_match.group(7)
        fuel_data['altn_dist'] = int(fuel_match.group(8))
        fuel_data['altn_time'] = float(fuel_match.group(9))
        fuel_data['altn_fuel'] = int(fuel_match.group(10))
        fuel_data['altn_planned_fuel'] = int(fuel_match.group(11))
        fuel_data['eldw'] = int(fuel_match.group(12))
        fuel_data['mldw'] = int(fuel_match.group(13))
        fuel_data['reserve_fuel'] = int(fuel_match.group(14))
        fuel_data['reserve_time'] = float(fuel_match.group(15))
        fuel_data['ezfw'] = int(fuel_match.group(16))
        fuel_data['mzfw'] = int(fuel_match.group(17))
        fuel_data['holding_dist'] = int(fuel_match.group(18))
        fuel_data['holding_time'] = float(fuel_match.group(19))
        fuel_data['epld'] = int(fuel_match.group(20))
        fuel_data['min_fuel_req'] = int(fuel_match.group(21))
        fuel_data['min_fuel_time'] = float(fuel_match.group(22))
        fuel_data['opnlwt'] = int(fuel_match.group(23))

    return fuel_data
    """esta parte aqui poderá ser subsituida por uma media do fuel flow or media
    historica do consumo de fuel do voo e model da craft. Inicialmente apenas averigua-se
    se valor é Osmar ou não dando assim permissão para seguir ou não"""

# Function to extract text from PDF
"""def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

# Get the PDF file path as input from the user
pdf_path = input("Enter the path to the PDF file: ")
pdf_text = extract_text_from_pdf(pdf_path)"""

if recognize_flight_plan(pdf_text):
    print("No fuel void or empty.")

    fuel_data = extract_fuel_data(pdf_text)

    # Check if any extracted number is less than or equal to 0
    has_invalid_numbers = any(value <= 0 for value in fuel_data.values())

    if has_invalid_numbers:
        print("Invalid numbers detected. Check your flight plan.")
    else:
        print("Valid flight plan.")

else:
    print("Not recognized as a flight plan.")

Recognized as a flight plan.
Recognized as a flight plan.
Valid flight plan.
